# 1. Setup

Clone repo, install dependencies and check PyTorch and GPU.

Alternatively, you can build the docker image or download our pre-built [image](https://hub.docker.com/repository/docker/sudochris/yolo6dpose/general) to reproduce the experiments

The code has been tested on Python 3.9.1 on both windows 11 and Ubunutu 20.04

In [ ]:
!git clone https://github.com/cviviers/YOLOv5-6D-Pose.git  # clone
!cd YOLOv5-6D-Pose
!pip install -r requirements.txt  # install

# (Optional) install dependecies for calculating the 6D pose of symmetric objects
# This requires python3.XX-dev (your version of python) in addition to a C++ compiler eg. sudo apt-get install python3.9-dev
!cd ./utils/
!python ./setup.py build_ext --inplace  # you will need to install cython # https://stackoverflow.com/questions/7508803/how-do-i-import-function-from-pyx-file-in-python

import torch
from IPython.display import clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

## Example project layout

```
├── data
│   ├── LINEMOD
│   ├── VOCdevkit
│   ├── Weights    
├── yolov5_6d_pose
│   ├── tutorial.ipynb
│   ├── train.py
│   ├── detect.py
│   ├── etc...
```

# 2. Inference

1. Download the pretrained weights from [google_drive](https://drive.google.com/drive/folders/11BW41xO3R1UBnc2Dx1xA3CPbYPGTrfHQ?usp=drive_link)
2. Extract them to a preferred location
3. `detect.py` runs inference on a variety of sources. In this example we included an image from the LINEMOD dataset

In [1]:
# Extract YOLOv5-6D pretrained weights, this includes the weights for the intialization and the fine-tuned models for each of the objects
!tar xf linemod_weights.tar -C ../weights

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'weights.tar'


In [ ]:
!python detect.py --weights path/to/cat/best.pt --img 640 --conf 0.25 --source results/images/example_cat.jpg --static-camera configs/linemod/linemod_camera.json --mesh-data path/to/cat.ply

# 3. Test
Test a model on [LINEMOD](https://arxiv.org/abs/1808.08319)

In [ ]:
# Download LINEMOD the original dataset
!wget -O LINEMOD.tar --no-check-certificate "https://onedrive.live.com/download?cid=05750EBEE1537631&resid=5750EBEE1537631%21135&authkey=AJRHFmZbcjXxTmI"
!tar xf LINEMOD.tar -C ../data

# Be sure to convert the data to our format focusing on the camera intrinsics
!python data_curation/convert_linemod.py --path_to_linemod ../data/LINEMOD

# Ulternatively, you can download the converted dataset from here: https://drive.google.com/drive/folders/13pkdF4KpqXWXbEIkiAcHoO-YUr_CWN-L?usp=drive_link
!tar xf LINEMOD_updated.tar -C ../data

## Test LINEMOD cat 
!python test.py --batch 1 --weights ../data/weights/linemod/cat/best.pt --data configs/linemod/cat.yaml --rect --cache

# 4. Train

Download [LINEMOD](https://bop.felk.cvut.cz/) and [VOCdevkit](http://host.robots.ox.ac.uk/pascal/VOC/index.html), start tensorboard/wnadb and train YOLOv5-6D (note actual training is typically much longer, around **300-6000 epochs**, depending on your dataset).

In [ ]:
# Download VOC, This can also be any other dataset you want to use for background images. We also use VOC for occlusion augmentation
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar  # Alternatively - http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar or https://drive.google.com/drive/folders/13pkdF4KpqXWXbEIkiAcHoO-YUr_CWN-L?usp=drive_link
!tar xf VOCtrainval_11-May-2012.tar -C ../data

In [ ]:
# Tensorboard (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
# Weights & Biases (optional)
%pip install -q wandb  

!wandb login  # use 'wandb disabled' or 'wandb enabled' to disable or enable

### Training configuration
```bash
python train.py --batch "batch size" --epochs "number of epochs" --cfg "model configuration" --hyp "hyper parameter configuration" --weights "starting weights" --data "data configuration" --rect --cache --optimizer Adam 
```

In [ ]:
# By default, the training script will download the COCO pretrained YOLOv5-v7 model. If this is no longer supported, you can download the weights from the YOLOv5-6D google drive (https://drive.google.com/drive/folders/11BW41xO3R1UBnc2Dx1xA3CPbYPGTrfHQ?usp=drive_link) and place them in the weights folder
!python train.py --batch 32 --epochs 5000 --cfg models/yolov5x_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights yolov5x.pt --data configs/linemod/benchvise.yaml --rect --cache --optimizer Adam 

# 5. Visualize
## Weights & Biases Logging

[Weights & Biases](https://www.wandb.com/) (W&B) is now integrated with YOLOv5 for real-time visualization and cloud logging of training runs. This allows for better run comparison and introspection, as well improved visibility and collaboration for teams. To enable W&B `pip install wandb`, and then train normally (you will be guided through setup on first use). 

During training you will see live updates at [https://wandb.ai/home](https://wandb.ai/home)

## Local Logging

All results are logged by default to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc. View train and test pngs to see labels, predictions and augmentation effects.

Training losses and performance metrics are also logged to [Tensorboard](https://www.tensorflow.org/tensorboard) and a custom `results.txt` logfile which is plotted as `results.png` (below) after training completes. 

# Environments

YOLOv5-6D may be run in any of the following up-to-date verified environments (with all dependencies including [CUDA](https://developer.nvidia.com/cuda)/[CUDNN](https://developer.nvidia.com/cudnn), [Python](https://www.python.org/) and [PyTorch](https://pytorch.org/) preinstalled):

- **Google Colab and Kaggle** notebooks with free GPU: <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a> <a href="https://www.kaggle.com/ultralytics/yolov5"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
- **Docker Image**. See [Docker Quickstart Guide](https://github.com/ultralytics/yolov5/wiki/Docker-Quickstart) <a href="https://hub.docker.com/r/ultralytics/yolov5"><img src="https://img.shields.io/docker/pulls/ultralytics/yolov5?logo=docker" alt="Docker Pulls"></a>
